## **Percobaan Klasifikasi dan Sortir Biji Kopi Peaberry, Longberry, Premium dan Defect dengan Menggunakan Deep Learning.**

#**DATA** 

## **Membaca dan mengakses data dari Google Drive**

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')  

## **Mengimpor package yang diperlukan**

In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time, os, copy, argparse
import multiprocessing
from torchsummary import summary
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
!pip install tensorboard

## **Launch kode program dengan environment variabel CUDA_LAUNCH_BLOCKING**

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## **Membuat variabel train, validasi dan test dari path direktori di Google Drive**



In [ ]:
# buat variabel train, validasi dan test dari path dataset
train_dir = '/content/drive/MyDrive/CFStar-Tech-800-2021/train'

val_dir = '/content/drive/MyDrive/CFStar-Tech-800-2021/val'

test_dir = '/content/drive/MyDrive/CFStar-Tech-800-2021/test'


## **Membuat variabel awal untuk percobaan**

In [ ]:
# Membuat variabel untuk banyak epoch
num_epochs = 25

# Membuat variabel untuk batch size
batch_size = 4

# variabel untuk jumlah kelas percobaan
num_classes = 4

# variabel untuk num_workers dimana memberitahukan berapa banyak sub-proses yang digunakan untuk memuat data
num_cpu = multiprocessing.cpu_count()

## **Menghitung nilai rata-rata (mean) dan standar deviasi (std) untuk dilakukan normalisasi gambar**

In [ ]:
# load dan menampilkan contoh gambar biji kopi yang akan dihitung

# Memuat gambar
path_base ='/content/drive/MyDrive/CFStar-Tech-800-2021/train'
img_path = path_base + '/longberry/1010.jpg'
img = Image.open(img_path)
img

In [ ]:
# Mengonversi gambar PIL ke numpy array
img_np = np.array(img)


# Membuat plot nilai setiap pixel gambar
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("nilai pixel")
plt.ylabel("frekuensi")
plt.title("distribusi dari setiap pixel")

In [ ]:
# membuat tansforms gambar ke Pytorch tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
  
# melakukan transform dari gambar PIL ke tensor 
img_tr = transform(img)

# mengonversi gambar tensor ke numpy array
img_np = np.array(img_tr)

# Membuat plot nilai setiap pixel gambar
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("nilai pixel")
plt.ylabel("frekuensi")
plt.title("distribusi dari setiap pixel")

In [ ]:
# menghitung nilai rata-rata (mean) dan standar deviasi (std)

# Gambar tensor
img_tr = transform(img)
  
# Menghitung mean dan std
mean, std = img_tr.mean([1,2]), img_tr.std([1,2])
  
# mencetak hasil kalkulasi
print("nilai rata-rata (mean) dan standar deviasi (std) sebelum normalisasi:")
print("rata-rata (mean) dari gambar      :", mean)
print("standar deviasi (std) dari gambar :", std)

In [ ]:
# membandingkan nilai sebelum normalisasi dengan setelah normalisasi

# membuat tansforms gambar ke Pytorch tensor dan normalisasi dari nilai mean dan std
transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
  
# gambar yang akan dinormalisasi
img_normalized = transform_norm(img)
  
# mengkonversi gambar normalisasi ke numpy array
img_np = np.array(img_normalized)
  
# Membuat plot nilai setiap pixel gambar
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("nilai pixel")
plt.ylabel("frekuensi")
plt.title("distribusi dari setiap pixel")

In [ ]:
# Menampilkan hasil gambar normalisasi

img_normalized = transform_norm(img)
  
# mengkonversi gambar ke numpy array
img_normalized = np.array(img_normalized)
  
# transpose dari shape (3,,) ke shape (,,3)
img_normalized = img_normalized.transpose(1, 2, 0)
  
# menampilkan hasil gambar normalisasi
plt.imshow(img_normalized)
plt.xticks([])
plt.yticks([])

In [ ]:
# menghitung nilai mean dan std setelah normalisasi dimana nilai mean yaitu 0.0 dan nilai std yaitu 1.0
  
# gambar yang sudah dinormalisasi
img_nor = transform_norm(img)

# menghitung nilai rata-rata dan standar deviasi
mean, std = img_nor.mean([1,2]), img_nor.std([1,2])
  
# mencetak hasil nilai rata-rata dan standar deviasi
print("nilai rata-rata dan standar deviasi dari normalisasi gambar:")
print("nilai rata-rata dari gambar        :", mean)
print("nilai standar deviasi dari gambar  :", std)

## **Membuat Data Transform**

In [ ]:
# membuat variabel nilai rata-rata (mean) dan standar deviasi (std) untuk digunakan dalam normalisasi gambar
mean = [0.7980, 0.7820, 0.7562]
std = [0.1441, 0.1729, 0.2341]

In [ ]:
# Membuat transform data gambar agar cocok saat dilatih
image_transforms = { 
    'train': transforms.Compose([
                                 transforms.Resize(size=(256, 256)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean, std)
                                 ]),
    'val': transforms.Compose([
                               transforms.Resize(size=(256, 256)),
                               transforms.ToTensor(),
                               transforms.Normalize(mean, std)
                               ]),
    'test': transforms.Compose([
                                transforms.Resize(size=(256, 256)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                                ]),
                    }

## **Ringkasan dan Visualisasi Dataset**

In [ ]:
# Memuat data dari folder
dataset = {
    'train': datasets.ImageFolder(root=train_dir, transform=image_transforms['train']),
    'val': datasets.ImageFolder(root=val_dir, transform=image_transforms['val']),
    'test': datasets.ImageFolder(root=test_dir, transform=image_transforms['test']),
}

In [ ]:
# memuat banyak kelas pada dataset
classes = dataset['train'].classes
print('banyak kelas pada dataset ini adalah', len(classes),'dan kelas pada dataset terdiri dari:', classes)

In [ ]:
# menghitung banyak data dari data latih, data validasi dan data uji
dataset_sizes = {
    'train':len(dataset['train']),
    'val':len(dataset['val']),
    'test':len(dataset['test'])
}

In [ ]:
# Mencetak banyak jumlah dataset
print("banyak data gambar pada data latih   :",dataset_sizes['train'])
print("banyak data gambar pada data validasi:", dataset_sizes['val'])
print("banyak data gambar pada data uji     :", dataset_sizes['test'])

In [ ]:
# membuat iterator untuk memuat dataset
dataloaders = {
    'train' : data.DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'val'   : data.DataLoader(dataset['val'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'test'  : data.DataLoader(dataset['test'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True)
}


print('banyak jumlah data latih per kelas adalah', len(dataloaders['train']), 'gambar')
print('banyak jumlah data validasi per kelas adalah', len(dataloaders['val']), 'gambar')
print('banyak jumlah data uji per kelas adalah', len(dataloaders['test']), 'gambar')

In [ ]:
# mengecek jumlah batch size, output warna dan jumlah ukuran gambar dalam bentuk pixel

images, labels = next(iter(dataloaders['train']))
print("jumlah kelas adalah", len(labels))
print("dimensi ukuran gambar adalah", images.shape)

In [ ]:
# visualisasi data gambar pada dataset

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = [0.7980, 0.7820, 0.7562]
    std = [0.1441, 0.1729, 0.2341]
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # jeda untuk memperbarui plot gambar

# Mengambil data gambar dari data latih
inputs, classes = next(iter(dataloaders['train']))
class_names = dataset['train'].classes
# membuat grid
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

#**PERCOBAAN 1**

## **Device default**

In [ ]:
# membuat device default sebagai gpu, jika ada
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images, labels = images.to(device), labels.to(device)

## **Percobaan 1: Resnet18**

In [ ]:
# percobaan 1: menggunakan pretrained model dengan arsitektur Resnet18

print("\nmemuat resnet18 \n")
model_ft = models.resnet18(pretrained=True)
    
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes )

# transfer model ke GPU
model_ft = model_ft.to(device)

for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad )
    
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer 
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Learning rate
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# mencetak ringkasan model
print('Model Summary:-\n')
summary(model_ft, input_size=(3, 256, 256))
print(model_ft)

In [ ]:
writer=SummaryWriter('/content/content/runs/summary_1')
writer.add_graph(model_ft, images)

In [ ]:
# Melatih model
print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    writer = SummaryWriter("content/runs/model_percobaan_1")
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
    
    writer.close()

    %load_ext tensorboard
    %tensorboard --logdir content

In [ ]:
# Training model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir content/runs
%reload_ext tensorboard

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

## **menyimpan model**

In [ ]:
# menyimpan model
PATH = 'model_percobaan_1.pth'
print("\nSaving the model...")
torch.save(model_ft, PATH)

## **Memprediksi dari data validasi**

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft)

## **memprediksi dari data uji**

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_ft)

## **memuat akurasi dari data uji**

In [ ]:
 def computeTestSetAccuracy(model, criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate((dataloaders['test'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average test loss and test accuracy
    avg_test_loss = test_loss/(dataset_sizes['test'])
    avg_test_acc = test_acc/(dataset_sizes['test'])

    print("\nTest accuracy : " + str(avg_test_acc))

In [ ]:
computeTestSetAccuracy(model_ft, criterion)

## **melakukan uji deteksi pada data uji**

In [ ]:
# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset['train'].class_to_idx.items()}
print(idx_to_class)

In [ ]:
def predict(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image

    '''
    
    transform = image_transforms['test']


    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 256, 256)
    
#    idx_to_class = {
#      "1":"defect",
#      "2":"longberry",
#      "3":"peaberry",
#      "4":"premium",
#    }

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)

        topk, topclass = ps.topk(3, dim=1)
        cls = idx_to_class[topclass.cpu().numpy()[0][0]]
        score = topk.cpu().numpy()[0][0]

        for i in range(3):
            print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().numpy()[0][i])

### Longberry

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1600.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1601.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1602.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1603.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1604.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1605.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1606.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1607.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1608.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1609.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1610.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1611.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1612.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1613.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1614.jpg'
print("Actual       : Longberry")
predict(model_ft, test)

### Defect

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1600.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1601.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1602.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1603.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1604.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1605.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1606.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1607.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1608.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1609.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1610.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1611.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1612.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1613.jpg'
print("Actual       : Defect")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1614.jpg'
print("Actual       : Defect")
predict(model_ft, test)

### Peaberry

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1604.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1600.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1601.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1602.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1603.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1605.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1606.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1607.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1608.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1609.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1610.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1611.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1612.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1613.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1614.jpg'
print("Actual       : Peaberry")
predict(model_ft, test)

### Premium

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1637.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1600.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1601.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1602.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1603.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1604.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1605.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1606.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1607.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1608.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1609.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1610.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1611.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1612.jpg'
print("Actual       : Premium")
predict(model_ft, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1613.jpg'
print("Actual       : Premium")
predict(model_ft, test)

## **Mengevaluasi model**

In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix) 

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
  
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm           = np.array([[325.,  32.,  11.,  32.],
                                               [  5., 322.,  10.,  63.],
                                               [  9.,   0., 278., 113.],
                                               [ 16.,   5.,   6., 373.]]),
                      normalize    = True,
                      target_names = ['defect', 'longberry', 'peaberry', 'premium'],
                      title        = "Confusion Matrix")

In [ ]:
#To get the per-class accuracy:

print(confusion_matrix.diag()/confusion_matrix.sum(1))

# **PERCOBAAN 2**

## **Device default**

In [ ]:
# membuat device default sebagai gpu, jika ada
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images, labels = images.to(device), labels.to(device)

## **Percobaan 2: MobileNetV2**

In [ ]:
# memuat pre-trained model yaitu MobileNetV2
print("\nLoading mobilenetv2 as feature extractor ...\n")
model_tl = models.mobilenet_v2(pretrained=True)    

# Freeze all the required layers (i.e except last conv block and fc layers)
for params in list(model_tl.parameters())[0:-5]:
  params.requires_grad = False

# Modify fc layers to match num_classes
num_ftrs=model_tl.classifier[-1].in_features

model_tl.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    )    

# Transfer the model to GPU
model_tl = model_tl.to(device)

# Print model summary
print('Model Summary:-\n')
for num, (name, param) in enumerate(model_tl.named_parameters()):
    print(num, name, param.requires_grad )

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer 
optimizer_tl = optim.Adam(model_tl.parameters(), lr=0.001)

# Learning rate decay
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_tl, step_size=7, gamma=0.1)

In [ ]:
# mencetak ringkasan model
print('Model Summary:-\n')
summary(model_tl, input_size=(3, 256, 256))
print(model_tl)

In [ ]:
writer=SummaryWriter('/content/content/runs/summary_model_2')
writer.add_graph(model_tl, images)

## **melatih model pada percobaan 2**

In [ ]:
# Melatih model
print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    writer = SummaryWriter("content/runs/model_percobaan_2")
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
    
    writer.close()

In [ ]:
# Training model
model_tl = train_model(model_tl, criterion, optimizer_tl, exp_lr_scheduler,
                       num_epochs=num_epochs)

## **menyimpan model**

In [ ]:
# menyimpan model
PATH = 'model_percobaan_2.pth'
print("\nSaving the model...")
torch.save(model_tl, PATH)

## **memprediksi dari data validasi**

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_tl)

## **memprediksi dari data uji**

In [ ]:
# Visualizing the model predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_tl)

## **memuat akurasi dari data uji**

In [ ]:
 def computeTestSetAccuracy(model, criterion):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate((dataloaders['test'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            print("Test Batch number: {:03d}, Test: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average test loss and test accuracy
    avg_test_loss = test_loss/(dataset_sizes['test'])
    avg_test_acc = test_acc/(dataset_sizes['test'])

    print("\nTest accuracy : " + str(avg_test_acc))

In [ ]:
computeTestSetAccuracy(model_tl, criterion)

## **melakukan uji deteksi pada data uji**

### Longberry

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1615.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1616.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1617.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1618.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1619.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1620.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1621.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1622.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1623.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1624.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1625.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1626.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1627.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1628.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/longberry/1629.jpg'
print('Actual       : Longberry')
predict(model_tl, test)

### Defect

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1615.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1616.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1617.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1618.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1619.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1620.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1621.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1622.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1623.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1624.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1625.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1626.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1627.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1628.jpg'
print('Actual       : Defect')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/defect/1629.jpg'
print('Actual       : Defect')
predict(model_tl, test)

### Peaberry

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1615.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1616.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1617.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1618.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1619.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1620.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1621.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1622.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1623.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1624.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1625.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1626.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1627.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1628.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/peaberry/1629.jpg'
print("Actual       : Peaberry")
predict(model_tl, test)

### Premium

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1614.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1615.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1616.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1617.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1618.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1619.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1620.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1621.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1622.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1623.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1624.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1625.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1626.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1627.jpg'
print('Actual       : Premium')
predict(model_tl, test)

In [ ]:
test = '/content/drive/MyDrive/CFStar-Tech-800-2021/test/premium/1628.jpg'
print('Actual       : Premium')
predict(model_tl, test)


## **mengevaluasi model**

In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_tl(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix) 

In [ ]:
plot_confusion_matrix(cm           = np.array([[323.,  22.,  27.,  28.],
                                               [ 18., 347.,  16.,  19.],
                                               [ 18.,   2., 326.,  54.],
                                               [ 55.,  19.,  21., 305.]]),
                      normalize    = True,
                      target_names = ['defect', 'longberry', 'peaberry', 'premium'],
                      title        = "Confusion Matrix")

In [ ]:
#To get the per-class accuracy:

print(confusion_matrix.diag()/confusion_matrix.sum(1))